<a href="https://colab.research.google.com/github/rizwannitk/airporttimezone/blob/main/timezone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To Get the timezone and offset of UTC time for Airports

In [ ]:
pip install tzwhere

In [ ]:
pip install timezonefinder

In [ ]:
pip install openpyxl

In [23]:
#import all libraries
import numpy as np
import pandas as pd
import datetime
import timezonefinder, pytz

tf = timezonefinder.TimezoneFinder()

In [6]:
#read the file from URL
url="https://ourairports.com/data/airports.csv"
df=pd.read_csv(url)

In [8]:
#take only airports having 3 digit IATA code , take requred columns,remove duplicates records
is_valid =  df['iata_code'].astype(str).map(len)==3
valid_airport=df[is_valid]
valid_airport_new=valid_airport[['iso_region','iso_country','iata_code','latitude_deg','longitude_deg']]
valid_airport_new=valid_airport_new.drop_duplicates()


In [ ]:
#check any duplicates
valid_airport_new['iata_code'].value_counts()

In [ ]:
#keep only one records for each IATA code
valid_airport_noduplicate = valid_airport_new.drop_duplicates(subset=['iata_code'])
valid_airport_noduplicate['iata_code'].value_counts()>1

In [ ]:
#add new columns
valid_airport_noduplicate["timezone"] = str(np.nan)
valid_airport_noduplicate["offsetseconds"]=np.nan

In [ ]:
valid_airport_noduplicate

In [41]:
valid_airport_noduplicate.dtypes

iso_region        object
iso_country       object
iata_code         object
latitude_deg     float64
longitude_deg    float64
timezone         float64
offsetseconds     object
dtype: object

In [43]:
#write function to update new columns
def foo(df1):
    tf = timezonefinder.TimezoneFinder()
    for index, row in df1.iterrows():
                                     #print(row['latitude_deg'], row['longitude_deg'])
                                     timezone_str = tf.certain_timezone_at(lat=row['latitude_deg'], lng=row['longitude_deg'])
                                     if timezone_str is None:
                                     #                        print ("Could not determine the time zone")
                                                              df1.at[index,'timezone'] = np.nan
                                     else:
                                          df1.at[index,'timezone'] = timezone_str
                                          try :
                                                timezone = pytz.timezone(timezone_str)
                                                dt = datetime.datetime.utcnow()
                                                df1.at[index,'offsetseconds'] =timezone.utcoffset(dt).total_seconds()
                                          except :
                                                  df1.at[index,'offsetseconds'] = np.nan
                                          
                                         # Display the current time in that time zone
                                         #timezone = pytz.timezone(timezone_str)
                                         #dt = datetime.datetime.utcnow()
                                         #print ("The time in %s is %s" % (timezone_str, dt + timezone.utcoffset(dt)))
    return(df1)

In [44]:
airport_timezone=foo(valid_airport_noduplicate)

In [ ]:
airport_timezone

In [46]:
#drop all NAN rows
airport_timezone=airport_timezone.dropna()

In [49]:
airport_timezone.to_excel('airport_timezone.xlsx') 